## Just testing library

Scroll down for real example

In [3]:
from langchain.document_loaders import TextLoader, PyPDFLoader
loader = PyPDFLoader('./original_data/MSL/MSL Notes_1.pdf')
documents = loader.load()

In [4]:
# print(documents[0].page_content)

In [6]:
from pydantic import BaseModel, Field

class Document(BaseModel):
    content: str
    source: dict

In [15]:
import os
import json
import random
import requests
import glob
from langchain.document_loaders import TextLoader, PyPDFLoader

no_langchaing_documents: list[Document] = []
chat_models = ["gpt-35-turbo-0613", "gpt-35-turbo-1106"] # "gpt-4-0613"

for path in glob.glob("../../data/MSL/*.pdf"):
    # print("Processing ", path)
    loader = PyPDFLoader(path)
    documents = loader.load()

    no_langchaing_documents.append(Document(content="\n".join([document.page_content for document in documents ]), source=documents[0].metadata))

system_prompts = [
    """
Please rephrase this piece of text.
Each entity, date, names should be replaces with different name. The new names are made up. Be creative.
Preserve names of the sections. Change the timestamps.
""", """
Please generate similar looking Medical Science Liaison (MSL) Notes.
Each named entity, date, names should be replaces with different name. The name is made up.
Preserve titles of the sections.
""", """
Please generate Medical Science Liaison (MSL) Notes based on the original document.
Each named entity, date, names should be replaces with Czech sounding name. The name is made up.
Preserve titles of the sections. 'Subject' introduce the topic of MSL - use made-up Czech name of drug. Rest of the document need to be in English
"""
]    

chat_model = random.choice(chat_models)          
api_root = f"http://localhost:7777/?chat_model={chat_model}"
doc = random.choice(no_langchaing_documents)
doc2 = random.choice(no_langchaing_documents)
system_prompt = random.choice(system_prompts)
resp = requests.post(
    url=f"{api_root}",
    json={
        "messages": [
            {
                "role": "system",
                "content": system_prompt,
            },
            {"role": "user", "content": f"""
This is example of document "{doc.content}".\n\n
Here is another example "{doc2.content}".\n\n
Merge the information from examples and create new version of document. Follow the same formatting.
Put @@ after you finish Closing Remarks section.
"""},
        ],
        "temperature": round(min(0.2 + random.random(), 1.0), 1),
        "max_tokens": 2000,
        "stop": ["@@"],
    },
)
print(system_prompt)
created_doc = resp.json()["choices"][0]["message"]["content"]
print(created_doc)


Please generate Medical Science Liaison (MSL) Notes based on the original document.
Each named entity, date, names should be replaces with Czech sounding name. The name is made up.
Preserve titles of the sections. 'Subject' introduce the topic of MSL - use made-up Czech name of drug. Rest of the document need to be in English

Subject:  Medical Science Liaison (MSL) Notes - Introduction of PulmoVive and ImmunoViva  
Date: December 15, 2024  
Provider:  Dr. Karel Novák  
Title:  Pulmonologist and Immunologist  
Institution:  BreatheWell Pulmonary Care Center and ImmunoWell Immunology Clinic  
Summary of Key Discussion Points:  
1. Introduction:  
• Introduced PulmoVive as an innovative therapy for COPD management and ImmunoViva as an advanced immunomodulatory therapy designed to enhance innate and adaptive immune responses.  
• Dr. Novák expressed cautious optimism for PulmoVive, acknowledging the need for advancements in COPD treatment. For ImmunoViva, Dr. Novák emphasized its unique 

In [35]:
print(documents[1].page_content)

• Discussed patient feedback indicating a preference for interventions directly addressing 
major cardiovascular ev ents.  
7. Emerging Research and Future Developments:  
• Highlighted ongoing research on Vitaligen's potential in preventing cardiovascular 
events.  
• Dr. Foster questioned the rationale behind expanding research without addressing 
existing concerns about clinical re levance.  
8. Competitive Landscape:  
• Compared Vitaligen to existing cardiovascular medications, with Dr. Foster expressing a 
preference for established therapies with proven outcomes.  
• Discussed challenges of introducing a new drug without clear advantages over existing 
treatments.  
9. Access and Reimbursement:  
• Dr. Foster expressed skepticism about the cost -effectiveness of Vitaligen, especially 
given existing therapeutic alternatives.  
• Shared concerns about potential financial burdens on patients and healthcare systems.  
10. Action Items and Follow -Up: 
• Dr. Foster did not express i

## Running with real data

Run on your local machine:
```bash
ssh -N -o ServerAliveInterval=10 -i Prague-bootcamp.pem -R 7777:localhost:7777  ubuntu@ec2-52-59-171-97.eu-central-1.compute.amazonaws.com -o "ProxyCommand=nc -X connect -x webproxy.merck.com:8080 %h %p"
```
Together with local version of Flask application:
```python
import requests
from flask import Flask
from flask import request, jsonify
from dotenv import load_dotenv

load_dotenv()
app = Flask(__name__)

api_root = "https://iapi-test.merck.com/gpt/v2"

@app.route("/", methods=["POST"])
def foo():
    data = request.get_json()
    print("INPUT:", end="")
    print(data)
    
    chat_model = request.args.get("chat_model", default="gpt-35-turbo-0301", type=str)
    resp = requests.post(
        url=f"{api_root}/{chat_model}/chat/completions",
        params={"api-version": "2023-05-15"},
        headers={"X-Merck-APIKey": "vtpqR2kYD1VAm0vPxvCWl6EnB4RILwPS"}, # genAI bootcamp key
        json=data,
    )
    print("OUTPUT:", end="")
    print(resp.text)
    return jsonify(resp.json())


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=7777)

```

In [9]:
from pydantic import BaseModel, Field

class Document(BaseModel):
    content: str
    source: dict

In [17]:
import os
import json
import random
import requests
import glob
from langchain.document_loaders import TextLoader, PyPDFLoader

no_langchaing_documents: list[Document] = []
chat_models = ["gpt-35-turbo-0613", "gpt-35-turbo-1106"] # "gpt-4-0613"

for path in glob.glob("../../data/MSL/*.pdf"):
    # print("Processing ", path)
    loader = PyPDFLoader(path)
    documents = loader.load()

    no_langchaing_documents.append(Document(content="\n".join([document.page_content for document in documents ]), source=documents[0].metadata))

system_prompts = [
    """
Please rephrase this piece of text.
Each entity, date, names should be replaces with different name. The new names are made up. Be creative.
Preserve names of the sections. Change the timestamps.
""", """
Please generate similar looking Medical Science Liaison (MSL) Notes.
Each named entity, date, names should be replaces with different name. The name is made up.
Preserve titles of the sections.
""", """
Please generate Medical Science Liaison (MSL) Notes based on the original document.
Each named entity, date, names should be replaces with Czech sounding name. The name is made up.
Preserve titles of the sections. 'Subject' introduce the topic of MSL - use made-up Czech name of drug. Rest of the document need to be in English
"""
]    

while len(no_langchaing_documents) < 1_000:
    print(len(no_langchaing_documents), end=",")
    chat_model = random.choice(chat_models)          
    api_root = f"http://localhost:7777/?chat_model={chat_model}"
    doc = random.choice(no_langchaing_documents)
    doc2 = random.choice(no_langchaing_documents)
    system_prompt = random.choice(system_prompts)
    resp = requests.post(
        url=f"{api_root}",
        json={
            "messages": [
                {
                    "role": "system",
                    "content": system_prompt,
                },
                {"role": "user", "content": f"""
    This is example of document "{doc.content}".\n\n
    Here is another example "{doc2.content}".\n\n
    Merge the information from examples and create new version of document. Follow the same formatting.
    Put @@ after you finish Closing Remarks section.
    """},
            ],
            "temperature": round(min(0.2 + random.random(), 1.0), 1),
            "max_tokens": 1000,
            "stop": ["@@"],
        },
    )
    try:
        created_doc = resp.json()["choices"][0]["message"]["content"]
        no_langchaing_documents.append(Document(content=created_doc, source={"source": chat_model, "original_doc": doc.source["source"], "system_prompt": system_prompt}))
    except KeyError:
        ...


35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [18]:
with open("synthetic_data/data_v3.json", "w") as f:
    json.dump([x.model_dump() for x in no_langchaing_documents], f)

In [10]:
with open("synthetic_data/data.json", "r") as f:
    no_langchaing_documents = json.load(f)
    no_langchaing_documents = [Document.parse_obj(x) for x in no_langchaing_documents]